# Imports

In [ ]:
from gstools import SRF, Gaussian, Exponential, Cubic, Spherical, TPLSimple, Matern, Linear
from gstools import Stable, Rational, Circular, HyperSpherical, SuperSpherical, JBessel, TPLGaussian, TPLExponential, TPLStable
import gstools
import numpy as np
from random import *
import random
import matplotlib as mpl
import matplotlib.pyplot as plt

# Data Simulation

### 1) Initialisations

Sizes:

- grid scale: $J$

- number of observations for the training step: $n = (1 + 2^J)^2$\
    $\Longrightarrow$ size of dyadic grid of observations ($n_x \times n_x$): $n_x = 1 + 2^J$
    
- total number of points in $S$: $N = N_x^2$\
    $\Longrightarrow$ size of "grid" of all points in $S$: $N_x = \text{step} \times (n_x - 1) + 1$, where step is a strictly positive constant describing the repartition of the observations $X'(\sigma_n)$ on the spatial domain
    
- number of observations for the prediction step: $d$

Parameters:

- parameters of the covariance function for Theoretical Kriging: $\theta$ and cov\_type for the type of covariance (Gaussian, Exponential, Cubic, Spherical, TPL, Matern, Stable, Rational)

- parameter for Empirical Kriging: $\nu$

- regularisation parameter for inverse matrix

In [ ]:
# Dyadic grid scale
J = 3

step = 5 # step = 5 (J = 3) ; step = 3 (J = 4)

# Number of observations for prediction
d = 10
# Seed for simulation (samples)
seed_d = 234

# Parameters of covariance function
ls = 10
theta = ls/2

# Type of covariance
cov_type = 1
# cov_type = 1: Gaussian model
# cov_type = 2: Exponential model
# cov_type = 3: Cubic model
# cov_type = 4: Spherical model
# cov_type = 5: TPL model
# cov_type = 6: Matern model
# cov_type = 7: Stable model
# cov_type = 8: Rational model

# Shape parameter for Matern, Stable and Rational models
shape_param = 3/2

# Parameter for Empirical Kriging
nu = 1.0

# Regularisation Parameter
reg = 1.25

### 2) Grid and Samples

**Training Sample**

Dyadic Grid with points locations $(\sigma_n) = (\sigma_1, \cdots, \sigma_n)$:

- grid in $[0, 1]^2$

- grid scale J

- number of points: $n = (1+2^J)^2$

$\Longrightarrow$ points\_train table of points locations of size $(n, 2)$

In [ ]:
def creation_dyadic_grid(J, step):
    nx = 1 + 2**J
    n = nx**2
   
    points_train = np.zeros((n, 2))
    for k in range(nx):
        for l in range(nx):
            i = k*(1+2**J)+l
            points_train[i][0] = int(k * step)
            points_train[i][1] = int(l * step)
    points_train = points_train.astype(int)
    
    return nx, n, points_train

**All Data**

Domain $S$:

- grid in $[0, 1]^2$

- N points locations (discretization of the spatial domain)

$\Longrightarrow$ points\_S table of points locations of size $(N, 2)$

In [ ]:
def creation_domain_S(J, nx, step):
    Nx = step * (nx-1) + 1
    N = Nx**2
    
    points_S = np.zeros((N, 2))
    k = 0
    for i in range(Nx):
        for j in range(Nx):
            points_S[k][0] = i
            points_S[k][1] = j
            k+=1
    points_S = points_S.astype(int)
    
    return Nx, N, points_S

**Prediction Sample**

Observation points $(s_d) = (s_1, \cdots, s_d)$:

- randomly selected in $[0, 1]^2$ (without duplicates)

- number of points: $d$

$\Longrightarrow$ points\_pred table of points locations of size $(d, 2)$

In [ ]:
def duplicat_sd(tab, a, b):
    n = tab.shape[0]
    for i in range(n):
        if tab[i][0] == a and tab[i][1] == b:
            return True
    return False


def creation_observations_points(Nx, d, seed):
    random.seed(seed)

    points_pred = - np.ones((d, 2))
    k = 0
    for i in range(d):
        ind_x = randint(0, Nx-1)
        ind_y = randint(0, Nx-1)
        while duplicat_sd(points_pred, ind_x, ind_y):
            ind_x = randint(0, Nx-1)
            ind_y = randint(0, Nx-1)
        points_pred[k][0] = ind_x
        points_pred[k][1] = ind_y
        k+=1
    points_pred = points_pred.astype(int)
    
    return points_pred

**Prediction Sample for Appendix C.2**

Extension to Different Configurations of the Observations’ Locations:

- Corner: When the major number of observations ($d_1 = 50$) are taken randomly in a small sub-region of the spatial domain $S$, defined as one of the corners of the domain.

- Ring: When the major number of observations ($d_c = 50$) are sampled randomly in a circle of center equal to the middle of the spatial domain $S$.

In [ ]:
def creation_observations_points_notreg(Nx, d1, d2, seed):
    random.seed(seed)

    d = d1 + d2
    
    points_pred = - np.ones((d, 2))
    k = 0
    for i in range(d1):
        ind_x = randint(0, int(Nx/3))
        ind_y = randint(0, int(Nx/3))
        while duplicat_sd(points_pred, ind_x, ind_y):
            ind_x = randint(0, int(Nx/3))
            ind_y = randint(0, int(Nx/3))
        points_pred[k][0] = ind_x
        points_pred[k][1] = ind_y
        k+=1
    
    for i in range(d2):
        ind_x = randint(0, Nx-1)
        ind_y = randint(0, Nx-1)
        while duplicat_sd(points_pred, ind_x, ind_y) and (ind_x <= Nx/3 and ind_y <= Nx/3):
            ind_x = randint(0, Nx-1)
            ind_y = randint(0, Nx-1)
        points_pred[k][0] = ind_x
        points_pred[k][1] = ind_y
        k+=1
    points_pred = points_pred.astype(int)
    
    return points_pred


def in_cercle(x, y, a, b, R):
    if (x-a)**2 + (y-b)**2 < R**2:
        return True
    else:
        return False


def creation_observations_points_ring(Nx, d, dc, seed):
    random.seed(seed)
    
    a = int(Nx/2)
    b = int(Nx/2)
    
    R = int(Nx/5)
    
    in_dc = 0
    
    points_pred = - np.ones((d, 2))
    
    k = 0
    
    while in_dc < dc:
        ind_x = randint(0, Nx-1)
        ind_y = randint(0, Nx-1)
        if in_cercle(ind_x, ind_y, a, b, R) and not duplicat_sd(points_pred, ind_x, ind_y):
            points_pred[k][0] = ind_x
            points_pred[k][1] = ind_y
            k+=1
            in_dc+=1
    
    d2 = d - dc
    
    for i in range(d2):
        ind_x = randint(0, Nx-1)
        ind_y = randint(0, Nx-1)
        while duplicat_sd(points_pred, ind_x, ind_y) and in_cercle(ind_x, ind_y, a, b, R):
            ind_x = randint(0, Nx-1)
            ind_y = randint(0, Nx-1)
        points_pred[k][0] = ind_x
        points_pred[k][1] = ind_y
        k+=1
    points_pred = points_pred.astype(int)
    
    return points_pred

### 3) Simulated Data

Simulate Gaussian random field using $\texttt{gstools}$:

- covariance model: Gaussian, Exponential, Cubic, Spherical, TPL

- dimension: $\mathbb{R}^2 \Longrightarrow dim = 2$

- variance: var $ = 1.0$

- length scale paramater: len_scale $ = 10$ (len_scale $ = 2 \theta$)

$\Longrightarrow$

- vector of observations $X$ used for Kriging: $X'(\sigma_n) = (X'_{\sigma_1}, \cdots, X'_{\sigma_n})$ for training data and $X(s_d) = (X_{s_1}, \cdots, X_{s_d})$ for prediction step

- matrix X\_tot of random field over all the spatial domain $S$

In [ ]:
def construct_datasets(data, locations):
    X = np.zeros((len(locations)))
    for i in range(len(locations)):
        ind_i = int(locations[i][0])
        ind_j = int(locations[i][1])
        X[i] = data[ind_i, ind_j]
    return(X)

In [ ]:
def simulation_field(Nx, cov_type, ls, seed, list_points_tot, list_points, shape_param):
    sx = sy = range(Nx)
    
    if cov_type == 1:
        model = Gaussian(dim=2, var=1, len_scale=ls)
    elif cov_type == 2:
        model = Exponential(dim=2, var=1, len_scale=ls)
    elif cov_type == 3:
        model = Cubic(dim=2, var=1, len_scale=ls)
    elif cov_type == 4:
        model = Spherical(dim=2, var=1, len_scale=ls)
    elif cov_type == 5:
        model = TPLSimple(dim=2, var=1, len_scale=ls)
    elif cov_type == 6:
        model = Matern(dim=2, var=1, len_scale=ls, nu=shape_param)
    elif cov_type == 7:
        model = Stable(dim=2, var=1, len_scale=ls, alpha=shape_param)
    elif cov_type == 8:
        model = Rational(dim=2, var=1, len_scale=ls, alpha=shape_param)
    
    srf = SRF(model, seed=seed)

    field = srf.structured([sx, sy]).T
    
    # Matrix with values of random field over all the spatial domain S
    X_tot = np.zeros((Nx, Nx))
    for s in list_points_tot:
        X_tot[s[0]][s[1]] = field[s[0]][s[1]]
    
    # Vector of observations
    X = construct_datasets(field, list_points)

    return X_tot, X

### 4) Plotting Field

Generalized function for plotting a field:

- random field (training data or predicition data)

- Kriging result (theoretical or empirical)

- quadratic difference (theoretical or empirical)

- covariance model used for repository

- theta used for file name

- grid scale $J$ used for repository

In [ ]:
def plotting_field(data_tot, list_points, params, type_dir, shape_param):
    
    plt.rc('xtick', labelsize=14) 
    plt.rc('ytick', labelsize=14)
    
    color = "red"
    
    # params[0]: random field, kriging result, quadratic difference
    # params[1] (if params[0] == 1 or if params[0] == 2): theoretical, empirical
    # params[2] (if params[0] == 0): training, prediction
    # params[3]: covariance model
    # params[4]: parameter theta of covariance model
    # params[5]: grid scale J
    
    if params[0] == 0: # random field
        if params[2] == 0:
            title = "Random Field for Training Step (with respective observed points)"
            name_fig = "RF_Training.png"
            color = "purple"
        else:
            title = "Random Field for Prediction Step (with respective observed points)"
            name_fig = "RF_Prediction.png"
    elif params[0] == 1: # kriging result
        if params[1] == 0:
            title = "Theoretical Kriging Result (with observed points)"
            name_fig = "Theoretical_Kriging.png"
        else:
            title = "Empirical Kriging Result (with observed points)"
            name_fig = "Empirical_Kriging.png"
    elif params[0] == 2: # quadratic difference
        if params[1] == 0:
            title = "Quadratic Difference for Theoretical Kriging (with observed points)"
            name_fig = "QD_Theoretical.png"
        else:
            title = "Quadratic Difference for Empirical Kriging (with observed points)"
            name_fig = "QD_Empirical.png"
    
    if params[3] == 1:
        directory = "./Results/" + type_dir + "/Gaussian/J = " + str(params[5]) + "/theta_" + str(params[4]) + "/"
    elif params[3] == 2:
        directory = "./Results/" + type_dir + "/Exponential/J = " + str(params[5]) + "/theta_" + str(params[4]) + "/"
    elif params[3] == 3:
        directory = "./Results/" + type_dir + "/Cubic/J = " + str(params[5]) + "/theta_" + str(params[4]) + "/"
    elif params[3] == 4:
        directory = "./Results/" + type_dir + "/Spherical/J = " + str(params[5]) + "/theta_" + str(params[4]) + "/"
    elif params[3] == 5:
        directory = "./Results/" + type_dir + "/TPL/J = " + str(params[5]) + "/theta_" + str(params[4]) + "/"
    elif params[3] == 6:
        directory = "./Results/" + type_dir + "/Matern/nu = " + str(shape_param) + "/J = " + str(params[5]) + "/theta_" + str(params[4]) + "/"
    elif params[3] == 7:
        directory = "./Results/" + type_dir + "/Stable/alpha = " + str(shape_param) + "/J = " + str(params[5]) + "/theta_" + str(params[4]) + "/"
    elif params[3] == 8:
        directory = "./Results/" + type_dir + "/Rational/alpha = " + str(shape_param) + "/J = " + str(params[5]) + "/theta_" + str(params[4]) + "/"
    
    fig = plt.figure(1, figsize=(7, 5))
    
    Nx = data_tot.shape[0]
    
    plt.imshow(data_tot, origin="lower", extent=[0,1,0,1])
    plt.scatter(list_points[:,0]/Nx, list_points[:,1]/Nx, c=color, marker='o', s = 40)
    
    bound_min = np.round(data_tot.min(), decimals=3)
    bound_max = np.round(data_tot.max(), decimals=3)
    cmap = mpl.cm.viridis
    norm = mpl.colors.Normalize(vmin=bound_min, vmax=bound_max)
    fig.colorbar(
        mpl.cm.ScalarMappable(cmap=cmap, norm=norm),
        extend='both',
        extendfrac='auto',
        spacing='uniform',
    )
    
    plt.savefig(directory + name_fig)
    plt.show()

# Theoretical Kriging

### True covariance function

- Gaussian model:
$$c : h \in [0,+\infty) \mapsto \exp\left(-h^2/\theta^2\right)$$

- Exponential model:
$$c : h \in [0,+\infty) \mapsto \exp\left(-h/\theta\right)$$

- Cubic:
$$c : h \in [0,+\infty) \mapsto \left(1 - \left(7 \frac{h^2}{\theta^2} - \frac{35}{4}\frac{h^3}{\theta^3} + \frac{7}{2}\frac{h^5}{\theta^5} - \frac{3}{4}\frac{h^7}{\theta^7}\right) \right)\,\mathbb{I}\left\{h \leq \theta\right\}$$

- Spherical model:
$$c : h \in [0,+\infty) \mapsto \left(1 - \left(\frac{3}{2}\frac{h}{\theta} - \frac{1}{2}\frac{h^3}{\theta^3}\right) \right)\,\mathbb{I}\left\{h \leq \theta\right\}$$

- Truncated Power Law model:
$$c : h \in [0,+\infty) \mapsto \left(1 - h/\theta \right)^{\frac{3}{2}}\,\mathbb{I}\left\{h \leq \theta\right\},$$
where $\theta \in \mathbb{R}_+^*$.

- Matern model with varying smoothness parameter $\nu$:
$$c : h \in [0,+\infty) \mapsto \frac{2^{1-\nu}}{\Gamma(\nu)} \left(\sqrt{2 \nu} \frac{h}{\theta}\right)^{\nu} K_{\nu}\left(\sqrt{2 \nu} \frac{h}{\theta}\right),$$
where $\nu$ is a smoothness parameter.

1: when $\nu = \frac{3}{2}$: $c(h) = \left(1 + \sqrt{3}\frac{h}{\theta}\right) \exp\left(- \sqrt{3}\frac{h}{\theta}\right)$

2: when $\nu = \frac{5}{2}$: $c(h) = \left(1 + \sqrt{5}\frac{h}{\theta} + \frac{5}{3} \left(\frac{h}{\theta}\right)^2\right) \exp\left(- \sqrt{5}\frac{h}{\theta}\right)$

- Stable model:
$$\rho : h \in [0,+\infty) \mapsto \exp\left(- \left(\frac{h}{\theta}\right)^{\alpha} \right),$$
where $\alpha \in (0, 2[$ is a shape parameter.

- Rational model:
$$\rho : h \in [0,+\infty) \mapsto \left(1 + \frac{1}{\alpha}\left(\frac{h}{\theta}\right)^2 \right)^{- \alpha},$$
where $\alpha > \frac{1}{2}$ is a shape parameter.

In [ ]:
def true_cov_func(h, theta, cov_type, shape_param):
    if cov_type == 1: # Gaussian
        return np.exp(-(h**2)/(theta**2))
    elif cov_type == 2: # exponential
        return np.exp(-h/theta)
    elif cov_type == 3: # cubic
        if h >= theta:
            return 0
        else:
            return 1 - ( 7*(h/theta)**2 - 8.75*(h/theta)**3 + 3.5*(h/theta)**5 - 0.75*(h/theta)**7 )
    elif cov_type == 4: # spherical
        if h >= theta:
            return 0
        else:
            return 1 - ( 1.5*(h/theta) - 0.5*(h/theta)**3 )
    elif cov_type == 5: # truncated power law
        if h >= theta:
            return 0
        else:
            nu = 1.5
            return (1 - (h/theta))**nu
    elif cov_type == 6: # Matern
        if shape_param == 3/2:
            return (1+np.sqrt(3)*(h/theta)) * np.exp(-np.sqrt(3)*(h/theta))
        elif shape_param == 5/2:
            return (1+np.sqrt(5)*(h/theta)+(5/3)*(h/theta)**2) * np.exp(-np.sqrt(5)*(h/theta))
    elif cov_type == 7: # stable
        return np.exp(-(h/theta)**shape_param)
    elif cov_type == 8: # rational
        return (1 + (1/shape_param)*(h/theta)**2)**(-shape_param)

### True covariance matrix at sampled points

$\Sigma(\mathbf{s}_d)=Var(\mathbf{X}(\mathbf{s}_d)) = (Cov(X_{s_i},X_{s_j}))_{i, j \leq d}$

In [ ]:
def construct_true_cov_mat(list_points, theta, cov_type, shape_param):
    d = len(list_points)
    true_cov_mat = np.zeros((d, d))
    for i in range(d):
        for j in range(i, d):
            h = np.sqrt((list_points[i][0] - list_points[j][0])**2 + (list_points[i][1] - list_points[j][1])**2)
            true_cov_mat[i][j] = true_cov_func(h, theta, cov_type, shape_param)
            true_cov_mat[j][i] = true_cov_mat[i][j]
    return true_cov_mat

### True covariance vector at sampled points

$\mathbf{c}_d(s)=(Cov(X_s,X_{s_1}),\; \ldots,\; Cov(X_s, X_{s_d}))$

In [ ]:
def construct_true_cov_vect(s, list_points, theta, cov_type, shape_param):
    d = len(list_points)
    true_cov_vect = np.zeros((d))
    for i in range(d):
        s_i = list_points[i]
        h = np.sqrt((s[0] - s_i[0])**2 + (s[1] - s_i[1])**2)
        true_cov_vect[i] = true_cov_func(h, theta, cov_type, shape_param)
    return true_cov_vect

### Theoretical Kriging at point $s \in S$

$$\widehat{X}_{s,\Lambda_d^*(s)}=\lambda_{1}^*(s)X_{s_1}+\ldots+\lambda_{d}^*(s)X_{s_d},$$
where $\Lambda_d^*(s)=\; \Sigma(\textbf{s}_d)^{-1}\textbf{c}_d(s) \in \mathbb{R}^d$ (see Lemma 1).

In [ ]:
def theoretical_kriging_s(s, data, list_points, precision_mat, theta, cov_type, shape_param):
    cov_vect_s = construct_true_cov_vect(s, list_points, theta, cov_type, shape_param)
    lambda_opt_s = np.dot(precision_mat, cov_vect_s)
    pred_opt_s = np.dot(np.transpose(data), lambda_opt_s)
    return pred_opt_s

### Theoretical Kriging over all domain $S$

Theoretical Kriging above for all $s \in S$.

In [ ]:
def theoretical_kriging_field(data, list_points_tot, n_S, list_points, theta, cov_type, shape_param):
    pred_opt = np.zeros((n_S, n_S))
    cov_mat = construct_true_cov_mat(list_points, theta, cov_type, shape_param)
    precision_mat = np.linalg.inv(cov_mat)
    for s in list_points_tot:
        pred_opt[s[0]][s[1]] = theoretical_kriging_s(s, data, list_points, precision_mat, theta, cov_type, shape_param)
    return pred_opt

# Empirical Kriging

## Nonparametric Covariance Estimation (Subsection 3.2)

The estimation of the covariance function, is based on a "large" number $n\geq 1$ of observations $\textbf{X'}(\sigma_n) := (X'_{\sigma_1},\; \ldots,\; X'_{\sigma_n})$

($\texttt{X_train}$ et $\texttt{points_train}$).

### Neighbourhoods $N(h)$

- $N(h) = \left\{(\sigma_i, \sigma_j), \, \|\sigma_i - \sigma_j\| = h, \, (i, j) \in \{ 1, \cdots, n \}^2\right\}$: set of pairs of sites that are at distance $h$ from one another

- $n_h = \vert N(h)\vert$: cardinality of $N(h)$ (notice that $n_0=n$)

Function returns:

- $\texttt{Nh} = \left\{X_{\sigma_i}X_{\sigma_j}, (\sigma_i, \sigma_j) \in N(h)\right\}$: already the product (used in the covariance estimation function) for the pairs of sites at distance $h$ from one another

- $n_h = \vert N(h) \vert$

In [ ]:
def N_h(dist, data, list_points):
    nh = 0
    Nh = list()
    n = len(list_points)
    for i in range(n):
        for j in range(n):
            h = np.sqrt((list_points[i][0] - list_points[j][0])**2 + (list_points[i][1] - list_points[j][1])**2)
            if h == dist:
                nh += 1
                Nh.append(data[i] * data[j])
    return Nh, nh

### List of distances $\mathcal{H}_n$

$\mathcal{H}_n = \{\vert\vert \sigma_i-\sigma_j\vert\vert:\; (i,j)\in \{1, \cdots, n \}^2\}$: set of observed lags (all possible lags observable on the dyadic grid).

In [ ]:
def list_distance(list_points):
    list_dist = list()
    n = len(list_points)
    for i in range(n):
        for j in range(i, n):
            h = np.sqrt((list_points[i][0] - list_points[j][0])**2 + (list_points[i][1] - list_points[j][1])**2)
            if not h in list_dist:
                list_dist.append(h)
    return list_dist

### Estimated Covariance Function (at Observed Lags)

#### Covariance function estimated for one lag $h \in \mathcal{H}_n$

$\widehat{c}(h)=\frac{1}{n_h}\sum\limits_{(\sigma_i, \sigma_j) \in N(h)}X_{\sigma_i}X_{\sigma_j}$.

Depends of maximum distance: if $n_h < \nu n$, $\widehat{c}(h)=0$ and $\widehat{c}(h)=\frac{1}{n_h}\sum\limits_{(\sigma_i, \sigma_j) \in N(h)}X_{\sigma_i}X_{\sigma_j}$, otherwise.

In [ ]:
def cov_est(dist, data, list_points, nu):
    Nh, nh = N_h(dist, data, list_points)
    n = data.shape[0]
    if nh < nu*n and dist != 0:
        return 0
    return 1/nh * np.sum(Nh)

#### List of all covariances for all observed lags

- allows to reduce computational time: estimation of covariance function at $h \in \mathcal{H}_n$ only once for each observed lag and keeping in memory each value $\widehat{c}(h)$.

- Depends of maximum distance: if $n_h < \nu n$, $\widehat{c}(h)=0$ and $\widehat{c}(h)=\frac{1}{n_h}\sum\limits_{(\sigma_i, \sigma_j) \in N(h)}X_{\sigma_i}X_{\sigma_j}$, otherwise.

In [ ]:
def construct_list_covariances(list_dist, data, list_points, nu):
    list_cov = np.zeros(len(list_dist))
    i = 0
    for h in list_dist:
        list_cov[i] = cov_est(h, data, list_points, nu)
        if list_cov[i] == 0:
            break;
        i+=1
    return list_cov

### Estimated Covariance Function for Unobserved Lags

Extrapolation of the empirical covariance function $\widehat{c}(h)$ at unobserved lags $h\in[0,\sqrt{2}-2^{-j_1}]\setminus \mathcal{H}_n$ by means of a piecewise constant estimator (the 1 NN estimator):
$\widehat{c}(h)=\widehat{c}(l_h)$, where $l_h=argmin_{l\in \mathcal{H}_n}\vert\vert h-l \vert\vert$ (breaking ties in an arbitrary fashion).

Function returns the Nearet Neighbour lag $l \in \mathcal{H}_n$ for an unobserved lag $h$.

In [ ]:
def nearest_neighb(dist, list_dist):
    min_h = np.abs(list_dist[0] - dist)
    argmin_h = list_dist[0]
    for i in range(1, len(list_dist)):
        if np.abs(list_dist[i]-dist) <= min_h:
            min_h = np.abs(list_dist[i]-dist)
            argmin_h = list_dist[i]
    return argmin_h

## Empirical Kriging (Subsection 3.3)

Equipped with the nonparametric covariance function estimator computed above on the training data, we now address the simple Kriging problem. Let $d\geq 1$ and consider arbitrary pairwise distinct sites $s_1,\; \ldots,\; s_d$ in $S=[0,1]^2$. The goal is to predict the value $X_s$ taken by $X$ at any site $s\in S$ based on $(X_{s_1},\; \ldots,\; X_{s_d})$.

Consider $\widehat{c}(\cdot)$, the estimator of the covariance function computed above, based on the $X'_{\sigma_i}$'s. From $\widehat{c}(\cdot)$, the covariance matrix $\Sigma(\mathbf{s}_d)$ and the covariance vector $\mathbf{c}_d(s)$ can be naturally estimated as follows:
$$\widehat{\mathbf{c}}_d(s) = \left( \widehat{c}(\vert\vert s-s_1\vert\vert,\; \ldots,\; \widehat{c}(\vert\vert s-s_d\vert\vert \right)\text{ for } s\in S,$$
and
$$\widehat{\Sigma}(\mathbf{s}_d) = \left( \widehat{c}(\vert\vert s_i-s_j\vert\vert )  \right)_{1\leq i,\; j\leq d}.$$

### Empirical covariance matrix at sampled points

$$\widehat{\Sigma}(\mathbf{s}_d) = \left( \widehat{c}(\vert\vert s_i-s_j\vert\vert )  \right)_{1\leq i,\; j\leq d}$$

In [ ]:
def construct_emp_cov_mat(list_cov, list_points, list_dist):
    d = len(list_points)
    emp_cov_mat = np.zeros((d, d))
    for i in range(d):
        for j in range(i, d):
            h = np.sqrt((list_points[i][0] - list_points[j][0])**2 + (list_points[i][1] - list_points[j][1])**2)
            if h in list_dist:
                ind_h = list_dist.index(h)
                emp_cov_mat[i, j] = list_cov[ind_h]
                emp_cov_mat[j, i] = emp_cov_mat[i, j]
            else:
                l = nearest_neighb(h, list_dist)
                ind_l = list_dist.index(l)
                emp_cov_mat[i, j] = list_cov[ind_l]
                emp_cov_mat[j, i] = emp_cov_mat[i, j]
    return emp_cov_mat

### Empirical covariance vector at sampled points

$$\widehat{\mathbf{c}}_d(s) = \left( \widehat{c}(\vert\vert s-s_1\vert\vert,\; \ldots,\; \widehat{c}(\vert\vert s-s_d\vert\vert \right)\text{ for } s\in S$$

In [ ]:
def construct_emp_cov_vect(s, list_cov, list_points, list_dist):
    d = len(list_points)
    emp_cov_vect = np.zeros((d))
    for i in range(d):
        s_i = list_points[i]
        h = np.sqrt((s[0] - s_i[0])**2 + (s[1] - s_i[1])**2)
        if h in list_dist:
            ind_h = list_dist.index(h)
            emp_cov_vect[i] = list_cov[ind_h]
        else:
            l = nearest_neighb(h, list_dist)
            ind_l = list_dist.index(l)
            emp_cov_vect[i] = list_cov[ind_l]
    return emp_cov_vect

### Empirical Kriging at point $s \in S$

$$f_{\widehat{\Lambda}_d}(s, \mathbf{X}(\mathbf{s}_d))=\langle\widehat{\Lambda}_d(s), \mathbf{X}(\mathbf{s}_d) \rangle=\; ^t\mathbf{X}(\mathbf{s}_d) \widehat{\Sigma}(\mathbf{s}_d)^{-1}\widehat{\mathbf{c}}_d(s),$$

where

$$\widehat{\Lambda}_d(s)=\; \widehat{\Sigma}(\mathbf{s}_d)^{-1}\widehat{\mathbf{c}}_d(s)$$

In [ ]:
def empirical_kriging_s(s, data, list_cov, list_points, list_dist, precision_mat):
    cov_vect_s = construct_emp_cov_vect(s, list_cov, list_points, list_dist)
    lambda_opt_s = np.dot(precision_mat, cov_vect_s)
    pred_opt_s = np.dot(np.transpose(data), lambda_opt_s)
    return pred_opt_s

### Empirical Kriging over all domain $S$

Empirical Kriging above for all $s \in S$: $f_{\widehat{\Lambda}_d}(s, \mathbf{X}(\mathbf{s}_d))$.

In [ ]:
def empirical_kriging_field(data, list_points_tot, n_S, list_cov, list_points, list_dist, reg):
    pred_opt = np.zeros((n_S, n_S))
    cov_mat = construct_emp_cov_mat(list_cov, list_points, list_dist)
    
    d = len(list_points)
    precision_mat = np.linalg.inv(cov_mat + reg*np.eye(d))
    
    for s in list_points_tot:
        pred_opt[s[0]][s[1]] = empirical_kriging_s(s, data, list_cov, list_points, list_dist, precision_mat)
    return pred_opt

# Results on Simulated Data

### Main Function

- 1: creation of domain $S$, dyadic grid of observations for training step, observation samples for prediction step

- 2: simulations of both training field $X$ and prediction field $X'$

- 3: Theoretical Kriging with MSE computation and Quadratic Difference field (MSE map)

- 4: Empirical Kriging with MSE computation and Quadratic Difference field (MSE map), with first, computation of list of observable distances on dyadic grid

$\Longrightarrow$ return: theoretical and empirical quadratic field, MSE of theoretical and empirical Kriging, list of sample observations

In [ ]:
def main_function(J, step, d, ls, theta, cov_type, seed_d, seed_sim_train, seed_sim_pred, nu, shape_param, reg, plot_res):
    # Creation data points
    nx, n, points_train = creation_dyadic_grid(J, step)
    Nx, N, points_S = creation_domain_S(J, nx, step)
    
    if plot_res:
        print("Spatial domain S = [0, 1]^2\n", "Size: N = Nx * Nx = ", Nx, " * ", Nx, " = ", N)
        print("Dyadic grid at scale J = ", J, "\n", "Size: n = nx * nx = ", nx, " * ", nx, " = ", n)
    
    points_pred = creation_observations_points(Nx, d, seed_d)
    
    if plot_res:
        print("Number of observations for the training step\n n = ", n)
        print("Number of observations for the prediction step\n d = ", d)
    
    # Simulations
    X_tot_train, X_train = simulation_field(Nx, cov_type, ls, seed_sim_train, points_S, points_train, shape_param)
    X_tot_pred, X_pred = simulation_field(Nx, cov_type, ls, seed_sim_pred, points_S, points_pred, shape_param)
    
    if plot_res:
        plotting_field(X_tot_train, points_train, params = [0, -1, 0, cov_type, theta, J], type_dir = "MCMC", shape_param = shape_param)
        plotting_field(X_tot_pred, points_pred, params = [0, -1, 1, cov_type, theta, J], type_dir = "MCMC", shape_param = shape_param)
    
    # Theoretical Kriging
    X_theo_kriging_pred = theoretical_kriging_field(X_pred, points_S, Nx, points_pred, theta, cov_type, shape_param)
    MSE_theo_pred = np.sum((X_theo_kriging_pred - X_tot_pred)**2 / N)
    if plot_res:
        plotting_field(X_theo_kriging_pred, points_pred, params = [1, 0, -1, cov_type, theta, J], type_dir = "MCMC", shape_param = shape_param)
    quad_diff_theo_pred = (X_tot_pred-X_theo_kriging_pred)**2
    
    # Empirical Kriging
    H_n_1 = list_distance(points_train)
    H_n = H_n_1.copy()
    H_n.sort()
    List_covariances_Hn = construct_list_covariances(H_n, X_train, points_train, nu)
    
    X_emp_kriging_pred = empirical_kriging_field(X_pred, points_S, Nx, List_covariances_Hn, points_pred, H_n, reg)
    MSE_emp_pred = np.sum((X_emp_kriging_pred - X_tot_pred)**2 / N)
    if plot_res:
        plotting_field(X_emp_kriging_pred, points_pred, params = [1, 1, -1, cov_type, theta, J], type_dir = "MCMC", shape_param = shape_param)
    quad_diff_emp_pred = (X_tot_pred-X_emp_kriging_pred)**2
    
    return quad_diff_theo_pred, MSE_theo_pred, quad_diff_emp_pred, MSE_emp_pred, points_pred

Repeat experiments for $100$ independent couple of simulations. For each experience, keep in memory both MSE and both MSE maps.

In [ ]:
# Dyadic grid scale
J = 3

step = 5 # step = 5 (J = 3) ; step = 3 (J = 4)

# Number of observations for prediction
d = 10
# Seed for simulation (samples)
seed_d = 234

# Parameters of covariance function
ls = 10
theta = ls/2

# Type of covariance
cov_type = 1
# cov_type = 1: Gaussian model
# cov_type = 2: Exponential model
# cov_type = 3: Cubic model
# cov_type = 4: Spherical model
# cov_type = 5: TPL model
# cov_type = 6: Matern model
# cov_type = 7: Stable model
# cov_type = 8: Rational model

# Shape parameter for Matern, Stable and Rational models
shape_param = 3/2

# Parameter for Empirical Kriging
nu = 1.0

# Regularisation Parameter
reg = 1.25

In [ ]:
n_exp = 100
    
list_theoretical_kriging = list()
list_theoretical_MSE = list()
list_empirical_kriging = list()
list_empirical_MSE = list()

plot_res = True

for exp in range(n_exp):
    if exp == 25 or exp == 50 or exp == 75 or exp == 99:
        print("exp nb: ", exp)
    seed_sim_train = exp
    seed_sim_pred = 1000 + exp
    
    quad_diff_theo, MSE_theo_exp, quad_diff_emp, MSE_emp_exp, points_pred = main_function(J, step, d, ls, theta, cov_type, seed_d, seed_sim_train, seed_sim_pred, nu, shape_param, reg, plot_res)
    
    list_theoretical_kriging.append(quad_diff_theo)
    list_theoretical_MSE.append(MSE_theo_exp)
    list_empirical_kriging.append(quad_diff_emp)
    list_empirical_MSE.append(MSE_emp_exp)
    
    plot_res = False
    
array_theoretical_kriging = np.array(list_theoretical_kriging)
array_theoretical_MSE = np.array(list_theoretical_MSE)
array_empirical_kriging = np.array(list_empirical_kriging)
array_empirical_MSE = np.array(list_empirical_MSE)
    
Quad_Diff_MEAN_theo = array_theoretical_kriging.mean(axis = 0)
plotting_field(Quad_Diff_MEAN_theo, points_pred, params = [2, 0, -1, cov_type, theta, J], type_dir = "MCMC", shape_param = shape_param)

Quad_Diff_MEAN_emp = array_empirical_kriging.mean(axis = 0)
plotting_field(Quad_Diff_MEAN_emp, points_pred, params = [2, 1, -1, cov_type, theta, J], type_dir = "MCMC", shape_param = shape_param)
    
print("Theoretical")
print("Mean MSE: ", array_theoretical_MSE.mean())
print("Standard Deviation MSE: ", array_theoretical_MSE.std())
    
print("Empirical")
print("Mean MSE: ", array_empirical_MSE.mean())
print("Standard Deviation MSE: ", array_empirical_MSE.std())

## Appendix C.2: Extension to Different Configurations of the Observations’ Locations:

### Regular

In [ ]:
# Dyadic grid scale
J = 4

step = 3 # step = 5 (J = 3) ; step = 3 (J = 4)

# Number of observations for prediction
d = 60
# Seed for simulation (samples)
seed_d = 234

# Parameters of covariance function
ls = 10
theta = ls/2

# Type of covariance
cov_type = 5
# cov_type = 1: Gaussian model
# cov_type = 2: Exponential model
# cov_type = 3: Cubic model
# cov_type = 4: Spherical model
# cov_type = 5: TPL model
# cov_type = 6: Matern model
# cov_type = 7: Stable model
# cov_type = 8: Rational model

# Shape parameter for Matern, Stable and Rational models
shape_param = 3/2

# Parameter for Empirical Kriging
nu = 1.0

# Regularisation Parameter
reg = 1.25

n_exp = 100
    
list_theoretical_kriging = list()
list_theoretical_MSE = list()
list_empirical_kriging = list()
list_empirical_MSE = list()

for exp in range(n_exp):
    if exp == 25 or exp == 50 or exp == 75 or exp == 99:
        print("exp nb: ", exp)
    seed_sim_train = exp
    seed_sim_pred = 1000 + exp
    
    quad_diff_theo, MSE_theo_exp, quad_diff_emp, MSE_emp_exp, points_pred = main_function(J, step, d, ls, theta, cov_type, seed_d, seed_sim_train, seed_sim_pred, nu, shape_param, reg, False)
    
    list_theoretical_kriging.append(quad_diff_theo)
    list_theoretical_MSE.append(MSE_theo_exp)
    list_empirical_kriging.append(quad_diff_emp)
    list_empirical_MSE.append(MSE_emp_exp)
    
array_theoretical_kriging = np.array(list_theoretical_kriging)
array_theoretical_MSE = np.array(list_theoretical_MSE)
array_empirical_kriging = np.array(list_empirical_kriging)
array_empirical_MSE = np.array(list_empirical_MSE)
    
Quad_Diff_MEAN_theo = array_theoretical_kriging.mean(axis = 0)
plotting_field(Quad_Diff_MEAN_theo, points_pred, params = [2, 0, -1, cov_type, theta, J], type_dir = "NotReg/Regular", shape_param = shape_param)

Quad_Diff_MEAN_emp = array_empirical_kriging.mean(axis = 0)
plotting_field(Quad_Diff_MEAN_emp, points_pred, params = [2, 1, -1, cov_type, theta, J], type_dir = "NotReg/Regular", shape_param = shape_param)
    
print("Theoretical")
print("Mean MSE: ", array_theoretical_MSE.mean())
print("Standard Deviation MSE: ", array_theoretical_MSE.std())
    
print("Empirical")
print("Mean MSE: ", array_empirical_MSE.mean())
print("Standard Deviation MSE: ", array_empirical_MSE.std())

### Corner

When the major number of observations ($d_1 = 50$) are taken randomly in a small sub-region of the spatial domain $S$, defined as one of the corners of the domain.

In [ ]:
def main_function_notreg(J, step, d1, d2, ls, theta, cov_type, seed_d, seed_sim_train, seed_sim_pred, nu, shape_param, reg, plot_res):
    # Creation data points
    nx, n, points_train = creation_dyadic_grid(J, step)
    Nx, N, points_S = creation_domain_S(J, nx, step)
    
    if plot_res:
        print("Spatial domain S = [0, 1]^2\n", "Size: N = Nx * Nx = ", Nx, " * ", Nx, " = ", N)
        print("Dyadic grid at scale J = ", J, "\n", "Size: n = nx * nx = ", nx, " * ", nx, " = ", n)
    
    points_pred_notreg = creation_observations_points_notreg(Nx, d1, d2, seed_d)
    
    if plot_res:
        print("Number of observations for the training step\n n = ", n)
        print("Number of observations for the prediction step\n d = ", d)
    
    # Simulations
    X_tot_train, X_train = simulation_field(Nx, cov_type, ls, seed_sim_train, points_S, points_train, shape_param)
    X_tot_pred, X_pred = simulation_field(Nx, cov_type, ls, seed_sim_pred, points_S, points_pred_notreg, shape_param)
    
    # Theoretical Kriging
    X_theo_kriging_pred = theoretical_kriging_field(X_pred, points_S, Nx, points_pred_notreg, theta, cov_type, shape_param)
    MSE_theo_pred = np.sum((X_theo_kriging_pred - X_tot_pred)**2 / N)
    if plot_res:
        print("MSE Theoretical Kriging: ", MSE_theo_pred)
        plotting_field(X_theo_kriging_pred, points_pred_notreg, params = [1, 0, -1, cov_type, theta, J], type_dir = "NotReg/Corner", shape_param = shape_param)
    quad_diff_theo_pred = (X_tot_pred-X_theo_kriging_pred)**2
    
    # Empirical Kriging
    H_n_1 = list_distance(points_train)
    H_n = H_n_1.copy()
    H_n.sort()
    List_covariances_Hn = construct_list_covariances(H_n, X_train, points_train, nu)
    
    X_emp_kriging_pred = empirical_kriging_field(X_pred, points_S, Nx, List_covariances_Hn, points_pred_notreg, H_n, reg)
    MSE_emp_pred = np.sum((X_emp_kriging_pred - X_tot_pred)**2 / N)
    if plot_res:
        print("MSE Empirical Kriging: ", MSE_emp_pred)
        plotting_field(X_emp_kriging_pred, points_pred_notreg, params = [1, 1, -1, cov_type, theta, J], type_dir = "NotReg/Corner", shape_param = shape_param)
    quad_diff_emp_pred = (X_tot_pred-X_emp_kriging_pred)**2
    
    return quad_diff_theo_pred, MSE_theo_pred, quad_diff_emp_pred, MSE_emp_pred, points_pred_notreg

In [ ]:
n_exp = 100
    
list_theoretical_kriging = list()
list_theoretical_MSE = list()
list_empirical_kriging = list()
list_empirical_MSE = list()

plot_res = True
    
for exp in range(n_exp):
    if exp == 25 or exp == 50 or exp == 75 or exp == 99:
        print("exp nb: ", exp)
    seed_sim_train = exp
    seed_sim_pred = 1000 + exp
    
    d1 = 50
    d2 = 10
                                                                            
    quad_diff_theo, MSE_theo_exp, quad_diff_emp, MSE_emp_exp, points_pred = main_function_notreg(J, step, d1, d2, ls, theta, cov_type, seed_d, seed_sim_train, seed_sim_pred, nu, shape_param, reg, plot_res)
    
    list_theoretical_kriging.append(quad_diff_theo)
    list_theoretical_MSE.append(MSE_theo_exp)
    list_empirical_kriging.append(quad_diff_emp)
    list_empirical_MSE.append(MSE_emp_exp)
    
    plot_res = False
    
array_theoretical_kriging = np.array(list_theoretical_kriging)
array_theoretical_MSE = np.array(list_theoretical_MSE)
array_empirical_kriging = np.array(list_empirical_kriging)
array_empirical_MSE = np.array(list_empirical_MSE)
    
Quad_Diff_MEAN_theo = array_theoretical_kriging.mean(axis = 0)
plotting_field(Quad_Diff_MEAN_theo, points_pred, params = [2, 0, -1, cov_type, theta, J], type_dir = "NotReg/Corner", shape_param = shape_param)

Quad_Diff_MEAN_emp = array_empirical_kriging.mean(axis = 0)
plotting_field(Quad_Diff_MEAN_emp, points_pred, params = [2, 1, -1, cov_type, theta, J], type_dir = "NotReg/Corner", shape_param = shape_param)
    
print("Theoretical")
print("Mean MSE: ", array_theoretical_MSE.mean())
print("Standard Deviation MSE: ", array_theoretical_MSE.std())
    
print("Empirical")
print("Mean MSE: ", array_empirical_MSE.mean())
print("Standard Deviation MSE: ", array_empirical_MSE.std())

### Ring

When the major number of observations ($d_c = 50$) are sampled randomly in a circle of center equal to the middle of the spatial domain $S$.

In [ ]:
def main_function_ring(J, step, d, dc, ls, theta, cov_type, seed_d, seed_sim_train, seed_sim_pred, nu, shape_param, reg, plot_res):
    # Creation data points
    nx, n, points_train = creation_dyadic_grid(J, step)
    Nx, N, points_S = creation_domain_S(J, nx, step)
    
    if plot_res:
        print("Spatial domain S = [0, 1]^2\n", "Size: N = Nx * Nx = ", Nx, " * ", Nx, " = ", N)
        print("Dyadic grid at scale J = ", J, "\n", "Size: n = nx * nx = ", nx, " * ", nx, " = ", n)
    
    points_pred_ring = creation_observations_points_ring(Nx, d, dc, seed_d)
    
    if plot_res:
        print("Number of observations for the training step\n n = ", n)
        print("Number of observations for the prediction step\n d = ", d)
    
    # Simulations
    X_tot_train, X_train = simulation_field(Nx, cov_type, ls, seed_sim_train, points_S, points_train, shape_param)
    X_tot_pred, X_pred = simulation_field(Nx, cov_type, ls, seed_sim_pred, points_S, points_pred_ring, shape_param)
    
    # Theoretical Kriging
    X_theo_kriging_pred = theoretical_kriging_field(X_pred, points_S, Nx, points_pred_ring, theta, cov_type, shape_param)
    MSE_theo_pred = np.sum((X_theo_kriging_pred - X_tot_pred)**2 / N)
    if plot_res:
        print("MSE Theoretical Kriging: ", MSE_theo_pred)
        plotting_field(X_theo_kriging_pred, points_pred_ring, params = [1, 0, -1, cov_type, theta, J], type_dir = "NotReg/Ring", shape_param = shape_param)
    quad_diff_theo_pred = (X_tot_pred-X_theo_kriging_pred)**2
    
    # Empirical Kriging
    H_n_1 = list_distance(points_train)
    H_n = H_n_1.copy()
    H_n.sort()
    List_covariances_Hn = construct_list_covariances(H_n, X_train, points_train, nu)
    
    X_emp_kriging_pred = empirical_kriging_field(X_pred, points_S, Nx, List_covariances_Hn, points_pred_ring, H_n, reg)
    MSE_emp_pred = np.sum((X_emp_kriging_pred - X_tot_pred)**2 / N)
    if plot_res:
        print("MSE Empirical Kriging: ", MSE_emp_pred)
        plotting_field(X_emp_kriging_pred, points_pred_ring, params = [1, 1, -1, cov_type, theta, J], type_dir = "NotReg/Ring", shape_param = shape_param)
    quad_diff_emp_pred = (X_tot_pred-X_emp_kriging_pred)**2
    
    return quad_diff_theo_pred, MSE_theo_pred, quad_diff_emp_pred, MSE_emp_pred, points_pred_ring

In [ ]:
n_exp = 100
    
list_theoretical_kriging = list()
list_theoretical_MSE = list()
list_empirical_kriging = list()
list_empirical_MSE = list()

plot_res = True
    
for exp in range(n_exp):
    if exp == 25 or exp == 50 or exp == 75 or exp == 99:
        print("exp nb: ", exp)
    seed_sim_train = exp
    seed_sim_pred = 1000 + exp
    
    d = 60
    dc = 50
                                                                            
    quad_diff_theo, MSE_theo_exp, quad_diff_emp, MSE_emp_exp, points_pred = main_function_ring(J, step, d, dc, ls, theta, cov_type, seed_d, seed_sim_train, seed_sim_pred, nu, shape_param, reg, plot_res)
    
    list_theoretical_kriging.append(quad_diff_theo)
    list_theoretical_MSE.append(MSE_theo_exp)
    list_empirical_kriging.append(quad_diff_emp)
    list_empirical_MSE.append(MSE_emp_exp)
    
    plot_res = False
    
array_theoretical_kriging = np.array(list_theoretical_kriging)
array_theoretical_MSE = np.array(list_theoretical_MSE)
array_empirical_kriging = np.array(list_empirical_kriging)
array_empirical_MSE = np.array(list_empirical_MSE)
    
Quad_Diff_MEAN_theo = array_theoretical_kriging.mean(axis = 0)
plotting_field(Quad_Diff_MEAN_theo, points_pred, params = [2, 0, -1, cov_type, theta, J], type_dir = "NotReg/Ring", shape_param = shape_param)

Quad_Diff_MEAN_emp = array_empirical_kriging.mean(axis = 0)
plotting_field(Quad_Diff_MEAN_emp, points_pred, params = [2, 1, -1, cov_type, theta, J], type_dir = "NotReg/Ring", shape_param = shape_param)
    
print("Theoretical")
print("Mean MSE: ", array_theoretical_MSE.mean())
print("Standard Deviation MSE: ", array_theoretical_MSE.std())
    
print("Empirical")
print("Mean MSE: ", array_empirical_MSE.mean())
print("Standard Deviation MSE: ", array_empirical_MSE.std())

## Covariance Function Estimation

In [ ]:
def cov_func(dist, data, list_points, list_dist, nu):
    if dist in list_dist:
        return cov_est(dist, data, list_points, nu)
    else:
        l = nearest_neighb(dist, list_dist)
        return cov_est(l, data, list_points, nu)

def main_function_cov_est(J, step, ls, theta, cov_type, seed_sim_train, nu, shape_param):
    # Creation data points
    nx, n, points_train = creation_dyadic_grid(J, step)
    Nx, N, points_S = creation_domain_S(J, nx, step)
    
    # Simulations
    X_tot_train, X_train = simulation_field(Nx, cov_type, ls, seed_sim_train, points_S, points_train, shape_param)
    
    # Empirical Kriging
    H_n_1 = list_distance(points_train)
    H_n = H_n_1.copy()
    H_n.sort()
    
    dist_min = min(H_n)
    dist_max = max(H_n)
    range_h = np.arange(dist_min, dist_max, 0.5)
    range_cov = np.zeros(len(range_h))
    range_cov_est = np.zeros(len(range_h))
    i = 0
    stop_est = False
    for h in range_h:
        range_cov[i] = true_cov_func(h, theta, cov_type, shape_param)
        if not stop_est:
            range_cov_est[i] = cov_func(h, X_train, points_train, H_n, nu)
        if range_cov_est[i] == 0:
            stop_est = True
        i+=1
    
    return range_h, range_cov, range_cov_est

In [ ]:
plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=14)

In [ ]:
# Dyadic grid scale
J = 3

step = 5 # step = 5 (J = 3) ; step = 3 (J = 4)

# Number of observations for prediction
d = 10
# Seed for simulation (samples)
seed_d = 234

# Parameters of covariance function
ls = 10
theta = ls/2

# Type of covariance
cov_type = 1
# cov_type = 1: Gaussian model
# cov_type = 2: Exponential model
# cov_type = 3: Cubic model
# cov_type = 4: Spherical model
# cov_type = 5: TPL model
# cov_type = 6: Matern model
# cov_type = 7: Stable model
# cov_type = 8: Rational model

# Shape parameter for Matern, Stable and Rational models
shape_param = 3/2

# Parameter for Empirical Kriging
nu = 1

n_exp = 100

list_range_cov = list()
list_range_cov_est = list()

for exp in range(n_exp):
    print("exp nb: ", exp)
    seed_sim_train = exp

    rangeh, rangecov, rangecovest = main_function_cov_est(J, step, ls, theta, cov_type, seed_sim_train, nu, shape_param)
    
    list_range_cov.append(rangecov)
    list_range_cov_est.append(rangecovest)

array_cov = np.array(list_range_cov)
array_cov_est = np.array(list_range_cov_est)

true_cov_mean = array_cov.mean(axis=0)
true_cov_std = array_cov.std(axis=0)

est_cov_mean = array_cov_est.mean(axis=0)
est_cov_std = array_cov_est.std(axis=0)

plt.plot(rangeh/40, true_cov_mean, "red")
plt.fill_between(rangeh/40, true_cov_mean - true_cov_std, true_cov_mean + true_cov_std,
                 color='red', alpha=0.2)

plt.plot(rangeh/40, est_cov_mean, "green")
plt.fill_between(rangeh/40, est_cov_mean - est_cov_std, est_cov_mean + est_cov_std,
                 color='green', alpha=0.2)

plt.xlabel("h", size = 18)
plt.ylabel("c(h)", size = 18)

plt.savefig("./Results/Covariance_Estimation/mean_cov_J3_theta_5_Gaussian.png")
plt.show()

zoom_val = 72

plt.plot((rangeh/40)[:zoom_val], true_cov_mean[:zoom_val], "red")
plt.fill_between((rangeh/40)[:zoom_val], (true_cov_mean - true_cov_std)[:zoom_val], (true_cov_mean + true_cov_std)[:zoom_val],
                 color='red', alpha=0.2)

plt.plot((rangeh/40)[:zoom_val], est_cov_mean[:zoom_val], "green")
plt.fill_between((rangeh/40)[:zoom_val], (est_cov_mean - est_cov_std)[:zoom_val], (est_cov_mean + est_cov_std)[:zoom_val],
                 color='green', alpha=0.2)

plt.xlabel("h", size = 18)
plt.ylabel("c(h)", size = 18)

plt.tight_layout()

plt.savefig("./Results/Covariance_Estimation/mean_cov_J3_theta_5_Gaussian_ZOOM.png")
plt.show()

# Appendix C.4: Anisotropic Covariance Function

Simulate an anisotropic covariance function with varying anisotropic ratios $\alpha$, where $\alpha = 1$ corresponds to the isotropy situation.

The covariance estimation is done thanks to a training dataset, observed at $n = 81$ sites ($J = 3$) and the prediction over the whole spatial domain is computed based on $d = 10$ observations. We fix the value of the correlation length at $\theta = 5$, and repeat the experiments for different degrees of anisotropy $\alpha \in \{0.9, 0.8, 0.7\}$.

In [ ]:
def simulation_field_anis(Nx, cov_type, ls, seed, list_points_tot, list_points, shape_param, anis):
    sx = sy = range(Nx)
    
    if cov_type == 1:
        model = Gaussian(dim=2, var=1, len_scale=ls, anis=anis)
    elif cov_type == 2:
        model = Exponential(dim=2, var=1, len_scale=ls, anis=anis)
    elif cov_type == 3:
        model = Cubic(dim=2, var=1, len_scale=ls, anis=anis)
    elif cov_type == 4:
        model = Spherical(dim=2, var=1, len_scale=ls, anis=anis)
    elif cov_type == 5:
        model = TPLSimple(dim=2, var=1, len_scale=ls, anis=anis)
    elif cov_type == 6:
        model = Matern(dim=2, var=1, len_scale=ls, nu=shape_param, anis=anis)
    
    srf = SRF(model, seed=seed)

    field = srf.structured([sx, sy]).T
    
    # Matrix with values of random field over all the spatial domain S
    X_tot = np.zeros((Nx, Nx))
    for s in list_points_tot:
        X_tot[s[0]][s[1]] = field[s[0]][s[1]]
    
    # Vector of observations
    X = construct_datasets(field, list_points)

    return X_tot, X, model

In [ ]:
def true_cov_func_anis(h, model_anis):
    return model_anis.covariance(h)

In [ ]:
def construct_true_cov_mat_anis(list_points, model):
    d = len(list_points)
    true_cov_mat = np.zeros((d, d))
    for i in range(d):
        for j in range(i, d):
            h = np.sqrt((list_points[i][0] - list_points[j][0])**2 + (list_points[i][1] - list_points[j][1])**2)
            true_cov_mat[i][j] = true_cov_func_anis(h, model)
            true_cov_mat[j][i] = true_cov_mat[i][j]
    return true_cov_mat

def construct_true_cov_vect_anis(s, list_points, model):
    d = len(list_points)
    true_cov_vect = np.zeros((d))
    for i in range(d):
        s_i = list_points[i]
        h = np.sqrt((s[0] - s_i[0])**2 + (s[1] - s_i[1])**2)
        true_cov_vect[i] = true_cov_func_anis(h, model)
    return true_cov_vect

def theoretical_kriging_s_anis(s, data, list_points, precision_mat, model):
    cov_vect_s = construct_true_cov_vect_anis(s, list_points, model)
    lambda_opt_s = np.dot(precision_mat, cov_vect_s)
    pred_opt_s = np.dot(np.transpose(data), lambda_opt_s)
    return pred_opt_s

def theoretical_kriging_field_anis(data, list_points_tot, n_S, list_points, model):
    pred_opt = np.zeros((n_S, n_S))
    cov_mat = construct_true_cov_mat_anis(list_points, model)
    precision_mat = np.linalg.inv(cov_mat)
    for s in list_points_tot:
        pred_opt[s[0]][s[1]] = theoretical_kriging_s_anis(s, data, list_points, precision_mat, model)
    return pred_opt

In [ ]:
def main_function_anis(J, step, d, ls, theta, cov_type, seed_d, seed_sim_train, seed_sim_pred, nu, shape_param, reg, plot_res, anis_param):
    # Creation data points
    nx, n, points_train = creation_dyadic_grid(J, step)
    Nx, N, points_S = creation_domain_S(J, nx, step)
    
    if plot_res:
        print("Spatial domain S = [0, 1]^2\n", "Size: N = Nx * Nx = ", Nx, " * ", Nx, " = ", N)
        print("Dyadic grid at scale J = ", J, "\n", "Size: n = nx * nx = ", nx, " * ", nx, " = ", n)
    
    points_pred = creation_observations_points(Nx, d, seed_d)
    
    if plot_res:
        print("Number of observations for the training step\n n = ", n)
        print("Number of observations for the prediction step\n d = ", d)
    
    # Simulations
    X_tot_train, X_train, model_anis = simulation_field_anis(Nx, cov_type, ls, seed_sim_train, points_S, points_train, shape_param, anis = anis_param)
    X_tot_pred, X_pred, model_anis = simulation_field_anis(Nx, cov_type, ls, seed_sim_pred, points_S, points_pred, shape_param, anis = anis_param)
    
    if plot_res:
        plotting_field(X_tot_train, points_train, params = [0, -1, 0, cov_type, theta, J], type_dir = "Anisotropic/" + str(anis_param), shape_param = shape_param)
        plotting_field(X_tot_pred, points_pred, params = [0, -1, 1, cov_type, theta, J], type_dir = "Anisotropic/" + str(anis_param), shape_param = shape_param)
    
    # Theoretical Kriging
    X_theo_kriging_pred = theoretical_kriging_field_anis(X_pred, points_S, Nx, points_pred, model_anis)
    MSE_theo_pred = np.sum((X_theo_kriging_pred - X_tot_pred)**2 / N)
    if plot_res:
        plotting_field(X_theo_kriging_pred, points_pred, params = [1, 0, -1, cov_type, theta, J], type_dir = "Anisotropic/" + str(anis_param), shape_param = shape_param)
    quad_diff_theo_pred = (X_tot_pred-X_theo_kriging_pred)**2
    
    # Empirical Kriging
    H_n_1 = list_distance(points_train)
    H_n = H_n_1.copy()
    H_n.sort()
    List_covariances_Hn = construct_list_covariances(H_n, X_train, points_train, nu)
    
    X_emp_kriging_pred = empirical_kriging_field(X_pred, points_S, Nx, List_covariances_Hn, points_pred, H_n, reg)
    MSE_emp_pred = np.sum((X_emp_kriging_pred - X_tot_pred)**2 / N)
    if plot_res:
        plotting_field(X_emp_kriging_pred, points_pred, params = [1, 1, -1, cov_type, theta, J], type_dir = "Anisotropic/" + str(anis_param), shape_param = shape_param)
    quad_diff_emp_pred = (X_tot_pred-X_emp_kriging_pred)**2
    
    return quad_diff_theo_pred, MSE_theo_pred, quad_diff_emp_pred, MSE_emp_pred, points_pred

In [ ]:
# Dyadic grid scale
J = 3

step = 5 # step = 5 (J = 3) ; step = 3 (J = 4)

# Number of observations for prediction
d = 10
# Seed for simulation (samples)
seed_d = 234

# Parameters of covariance function
ls = 20
theta = ls/2

# Type of covariance
cov_type = 1
# cov_type = 1: Gaussian model
# cov_type = 2: Exponential model
# cov_type = 3: Cubic model
# cov_type = 4: Spherical model
# cov_type = 5: TPL model
# cov_type = 6: Matern model

# Shape parameter for Matern model
shape_param = 3/2

# Anisotropy parameter
anis_param = 0.9

# Parameter for Empirical Kriging
nu = 1.0

# Regularisation Parameter
reg = 1.25

In [ ]:
n_exp = 100
    
list_theoretical_kriging = list()
list_theoretical_MSE = list()
list_empirical_kriging = list()
list_empirical_MSE = list()

plot_res = True

for exp in range(n_exp):
    if exp == 25 or exp == 50 or exp == 75 or exp == 99:
        print("exp nb: ", exp)
    seed_sim_train = exp
    seed_sim_pred = 1000 + exp
    
    quad_diff_theo, MSE_theo_exp, quad_diff_emp, MSE_emp_exp, points_pred = main_function_anis(J, step, d, ls, theta, cov_type, seed_d, seed_sim_train, seed_sim_pred, nu, shape_param, reg, plot_res, anis_param)
    
    list_theoretical_kriging.append(quad_diff_theo)
    list_theoretical_MSE.append(MSE_theo_exp)
    list_empirical_kriging.append(quad_diff_emp)
    list_empirical_MSE.append(MSE_emp_exp)
    
    plot_res = False
    
array_theoretical_kriging = np.array(list_theoretical_kriging)
array_theoretical_MSE = np.array(list_theoretical_MSE)
array_empirical_kriging = np.array(list_empirical_kriging)
array_empirical_MSE = np.array(list_empirical_MSE)
    
Quad_Diff_MEAN_theo = array_theoretical_kriging.mean(axis = 0)
plotting_field(Quad_Diff_MEAN_theo, points_pred, params = [2, 0, -1, cov_type, theta, J], type_dir = "Anisotropic/" + str(anis_param), shape_param = shape_param)

Quad_Diff_MEAN_emp = array_empirical_kriging.mean(axis = 0)
plotting_field(Quad_Diff_MEAN_emp, points_pred, params = [2, 1, -1, cov_type, theta, J], type_dir = "Anisotropic/" + str(anis_param), shape_param = shape_param)
    
print("Theoretical")
print("Mean MSE: ", array_theoretical_MSE.mean())
print("Standard Deviation MSE: ", array_theoretical_MSE.std())
    
print("Empirical")
print("Mean MSE: ", array_empirical_MSE.mean())
print("Standard Deviation MSE: ", array_empirical_MSE.std())

# Appendix C.5: Irregular Grid

The irregular grids are generated from regular grids using Bernoulli sampling, with varying probability $p$ of observing a spatial site.

### Preferential Sampling

In [ ]:
def creation_irregular_grid(J, step, p, seed_irr_grid):
    np.random.seed(seed=seed_irr_grid)
    nx = 1 + 2**J
    n = nx**2
   
    points_train_irregular = list()
    for k in range(nx):
        for l in range(nx):
            if np.random.random() <= p:
                points_train_irregular.append([int(k*step), int(l*step)])
    points_train_irregular = np.array(points_train_irregular)
    points_train_irregular = points_train_irregular.astype(int)
    
    n = points_train_irregular.shape[0]
    return nx, n, points_train_irregular

In [ ]:
def cov_func(dist, data, list_points, list_dist, nu):
    if dist in list_dist:
        return cov_est(dist, data, list_points, nu)
    else:
        l = nearest_neighb(dist, list_dist)
        return cov_est(l, data, list_points, nu)

def main_function_cov_est_irregular(J, step, p, ls, theta, cov_type, seed_sim_train, seed_irr_grid, nu, nu_matern):
    # Creation data points
    nx, n, points_train = creation_irregular_grid(J, step, p, seed_irr_grid)
    Nx, N, points_S = creation_domain_S(J, nx, step)
    
    # Simulations
    X_tot_train, X_train = simulation_field(Nx, cov_type, ls, seed_sim_train, points_S, points_train, nu_matern)
    
    # Empirical Kriging
    H_n_1 = list_distance(points_train)
    H_n = H_n_1.copy()
    H_n.sort()
    
    dist_min = min(H_n)
    dist_max = max(H_n)
    range_h = np.arange(dist_min, dist_max, 0.5)
    range_cov = np.zeros(len(range_h))
    range_cov_est = np.zeros(len(range_h))
    i = 0
    stop_est = False
    for h in range_h:
        range_cov[i] = true_cov_func(h, theta, cov_type, nu_matern)
        if not stop_est:
            range_cov_est[i] = cov_func(h, X_train, points_train, H_n, nu)
        if range_cov_est[i] == 0:
            stop_est = True
        i+=1
    
    return range_h, range_cov, range_cov_est

In [ ]:
plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=14)

In [ ]:
# Dyadic grid scale
J = 3

step = 5 # step = 5 (J = 3) ; step = 3 (J = 4)

# Parameter of Bernoulli sampling
p = 0.8
# Seed for irregular grid
seed_irr_grid = 56

# Number of observations for prediction
d = 10
# Seed for simulation (samples)
seed_d = 234

# Parameters of covariance function
ls = 10
theta = ls/2

# Type of covariance
cov_type = 1
# cov_type = 1: Gaussian model
# cov_type = 2: Exponential model
# cov_type = 3: Cubic model
# cov_type = 4: Spherical model
# cov_type = 5: TPL model
# cov_type = 6: Matern model

# Shape parameter for Matern model
shape_param = 3/2

# Parameter for Empirical Kriging
nu = 1

n_exp = 100

list_range_cov = list()
list_range_cov_est = list()

for exp in range(n_exp):
    print("exp nb: ", exp)
    seed_sim_train = exp

    rangeh, rangecov, rangecovest = main_function_cov_est_irregular(J, step, p, ls, theta, cov_type, seed_sim_train, seed_irr_grid, nu, shape_param)
    
    list_range_cov.append(rangecov)
    list_range_cov_est.append(rangecovest)

array_cov = np.array(list_range_cov)
array_cov_est = np.array(list_range_cov_est)

true_cov_mean = array_cov.mean(axis=0)
true_cov_std = array_cov.std(axis=0)

est_cov_mean = array_cov_est.mean(axis=0)
est_cov_std = array_cov_est.std(axis=0)

plt.plot(rangeh/40, true_cov_mean, "red")
plt.fill_between(rangeh/40, true_cov_mean - true_cov_std, true_cov_mean + true_cov_std,
                 color='red', alpha=0.2)

plt.plot(rangeh/40, est_cov_mean, "green")
plt.fill_between(rangeh/40, est_cov_mean - est_cov_std, est_cov_mean + est_cov_std,
                 color='green', alpha=0.2)

plt.xlabel("h", size = 18)
plt.ylabel("c(h)", size = 18)

plt.tight_layout()

plt.savefig("./Results/Covariance_Estimation/mean_cov_J3_theta_5_Gaussian_irr_grid_0.8.png")
plt.show()

zoom_val = 72

plt.plot((rangeh/40)[:zoom_val], true_cov_mean[:zoom_val], "red")
plt.fill_between((rangeh/40)[:zoom_val], (true_cov_mean - true_cov_std)[:zoom_val], (true_cov_mean + true_cov_std)[:zoom_val],
                 color='red', alpha=0.2)

plt.plot((rangeh/40)[:zoom_val], est_cov_mean[:zoom_val], "green")
plt.fill_between((rangeh/40)[:zoom_val], (est_cov_mean - est_cov_std)[:zoom_val], (est_cov_mean + est_cov_std)[:zoom_val],
                 color='green', alpha=0.2)

plt.xlabel("h", size = 18)
plt.ylabel("c(h)", size = 18)

plt.tight_layout()

plt.savefig("./Results/Covariance_Estimation/mean_cov_J3_theta_5_Gaussian_irr_grid_0.8_ZOOM.png")
plt.show()

In [ ]:
def main_function_irregular(J, step, p, d, ls, theta, cov_type, seed_d, seed_irr_grid, seed_sim_train, seed_sim_pred, nu, nu_matern, reg, plot_res):
    # Creation data points
    nx, n, points_train = creation_irregular_grid(J, step, p, seed_irr_grid)
    Nx, N, points_S = creation_domain_S(J, nx, step)
    
    if plot_res:
        print("Spatial domain S = [0, 1]^2\n", "Size: N = Nx * Nx = ", Nx, " * ", Nx, " = ", N)
        print("Dyadic grid at scale J = ", J, "\n", "Size: n = nx * nx = ", nx, " * ", nx, " = ", n)
    
    points_pred = creation_observations_points(Nx, d, seed_d)
    
    if plot_res:
        print("Number of observations for the training step\n n = ", n)
        print("Number of observations for the prediction step\n d = ", d)
    
    # Simulations
    X_tot_train, X_train = simulation_field(Nx, cov_type, ls, seed_sim_train, points_S, points_train, nu_matern)
    X_tot_pred, X_pred = simulation_field(Nx, cov_type, ls, seed_sim_pred, points_S, points_pred, nu_matern)
    
    """if plot_res:
        plotting_field(X_tot_train, points_train, params = [0, -1, 0, cov_type, theta, J], type_dir = "MCMC", nu_matern = nu_matern)
        plotting_field(X_tot_pred, points_pred, params = [0, -1, 1, cov_type, theta, J], type_dir = "MCMC", nu_matern = nu_matern)
    """
    # Theoretical Kriging
    X_theo_kriging_pred = theoretical_kriging_field(X_pred, points_S, Nx, points_pred, theta, cov_type, nu_matern)
    MSE_theo_pred = np.sum((X_theo_kriging_pred - X_tot_pred)**2 / N)
    """if plot_res:
        plotting_field(X_theo_kriging_pred, points_pred, params = [1, 0, -1, cov_type, theta, J], type_dir = "MCMC", nu_matern = nu_matern)
    """
    quad_diff_theo_pred = (X_tot_pred-X_theo_kriging_pred)**2
    
    # Empirical Kriging
    H_n_1 = list_distance(points_train)
    H_n = H_n_1.copy()
    H_n.sort()
    List_covariances_Hn = construct_list_covariances(H_n, X_train, points_train, nu)
    
    X_emp_kriging_pred = empirical_kriging_field(X_pred, points_S, Nx, List_covariances_Hn, points_pred, H_n, reg)
    MSE_emp_pred = np.sum((X_emp_kriging_pred - X_tot_pred)**2 / N)
    """if plot_res:
        plotting_field(X_emp_kriging_pred, points_pred, params = [1, 1, -1, cov_type, theta, J], type_dir = "MCMC", nu_matern = nu_matern)
    """
    quad_diff_emp_pred = (X_tot_pred-X_emp_kriging_pred)**2
    
    return quad_diff_theo_pred, MSE_theo_pred, quad_diff_emp_pred, MSE_emp_pred, points_pred

In [ ]:
# Dyadic grid scale
J = 3

step = 5 # step = 5 (J = 3) ; step = 3 (J = 4)

# Parameter of Bernoulli sampling
p = 0.8
# Seed for irregular grid
seed_irr_grid = 56

# Number of observations for prediction
d = 10
# Seed for simulation (samples)
seed_d = 234

# Parameters of covariance function
ls = 10
theta = ls/2

# Type of covariance
cov_type = 1
# cov_type = 1: Gaussian model
# cov_type = 2: Exponential model
# cov_type = 3: Cubic model
# cov_type = 4: Spherical model
# cov_type = 5: TPL model
# cov_type = 6: Matern model

# Shape parameter for Matern model
shape_param = 3/2

# Parameter for Empirical Kriging
nu = 1.0

# Regularisation Parameter
reg = 1.25

In [ ]:
n_exp = 100
    
list_theoretical_kriging = list()
list_theoretical_MSE = list()
list_empirical_kriging = list()
list_empirical_MSE = list()

plot_res = True

for exp in range(n_exp):
    if exp == 25 or exp == 50 or exp == 75 or exp == 99:
        print("exp nb: ", exp)
    seed_sim_train = exp
    seed_sim_pred = 1000 + exp
    
    quad_diff_theo, MSE_theo_exp, quad_diff_emp, MSE_emp_exp, points_pred = main_function_irregular(J, step, p, d, ls, theta, cov_type, seed_d, seed_irr_grid, seed_sim_train, seed_sim_pred, nu, nu_matern, reg, plot_res)
    
    list_theoretical_kriging.append(quad_diff_theo)
    list_theoretical_MSE.append(MSE_theo_exp)
    list_empirical_kriging.append(quad_diff_emp)
    list_empirical_MSE.append(MSE_emp_exp)
    
    plot_res = False
    
array_theoretical_kriging = np.array(list_theoretical_kriging)
array_theoretical_MSE = np.array(list_theoretical_MSE)
array_empirical_kriging = np.array(list_empirical_kriging)
array_empirical_MSE = np.array(list_empirical_MSE)
    
Quad_Diff_MEAN_theo = array_theoretical_kriging.mean(axis = 0)
plotting_field(Quad_Diff_MEAN_theo, points_pred, params = [2, 0, -1, cov_type, theta, J], type_dir = "Irregular_Grid/p = 0.8", nu_matern = nu_matern)

Quad_Diff_MEAN_emp = array_empirical_kriging.mean(axis = 0)
plotting_field(Quad_Diff_MEAN_emp, points_pred, params = [2, 1, -1, cov_type, theta, J], type_dir = "Irregular_Grid/p = 0.8", nu_matern = nu_matern)
    
print("Theoretical")
print("Mean MSE: ", array_theoretical_MSE.mean())
print("Standard Deviation MSE: ", array_theoretical_MSE.std())
    
print("Empirical")
print("Mean MSE: ", array_empirical_MSE.mean())
print("Standard Deviation MSE: ", array_empirical_MSE.std())